In [20]:
import sqlite3 as sql3
import pandas as pd
import csv

In [67]:
csv_escolas = 'MICRODADOS_ENEM_ESCOLA.csv'
csv_idhm_municipios = 'idhm-municipios.csv'
db_file_path = 'tp2.db'
tabela_escolas = 'escola'
tabela_idhm_municipios = 'idhm'
try:
    # Jogando csv em um dataframe pandas
    df_escolas = pd.read_csv(csv_escolas, encoding='latin-1', sep=';')
    create_table_query = f"""
    CREATE TABLE {tabela_escolas} (
        NU_ANO INTEGER NOT NULL,
        CO_UF_ESCOLA INTEGER,
        SG_UF_ESCOLA CHAR(2),
        CO_MUNICIPIO_ESCOLA INTEGER,
        NO_MUNICIPIO_ESCOLA TEXT,
        CO_ESCOLA_EDUCACENSO INTEGER,
        NO_ESCOLA_EDUCACENSO TEXT,
        TP_DEPENDENCIA_ADM_ESCOLA CHAR(1),
        TP_LOCALIZACAO_ESCOLA CHAR(1),
        NU_MATRICULAS INTEGER,
        NU_PARTICIPANTES_NEC_ESP INTEGER,
        NU_PARTICIPANTES INTEGER,
        NU_TAXA_PARTICIPACAO FLOAT,
        NU_MEDIA_CN FLOAT,
        NU_MEDIA_CH FLOAT,
        NU_MEDIA_LP FLOAT,
        NU_MEDIA_MT FLOAT,
        NU_MEDIA_RED FLOAT,
        NU_MEDIA_OBJ FLOAT,
        NU_MEDIA_TOT FLOAT,
        INSE TEXT,
        PC_FORMACAO_DOCENTE FLOAT,
        NU_TAXA_PERMANENCIA FLOAT,
        NU_TAXA_APROVACAO FLOAT,
        NU_TAXA_REPROVACAO FLOAT,
        NU_TAXA_ABANDONO FLOAT,
        PORTE_ESCOLA TEXT,
        CONSTRAINT check_TP_DEPENDENCIA_ADM_ESCOLA CHECK (TP_DEPENDENCIA_ADM_ESCOLA IN ('1','2','3','4','5','6','7','8','9')),
        CONSTRAINT check_TP_LOCALIZACAO_ESCOLA CHECK (TP_LOCALIZACAO_ESCOLA IN ('1','2')),
        CONSTRAINT pk_escola PRIMARY KEY(NU_ANO, CO_ESCOLA_EDUCACENSO)
    );
    """
    df_idhm_municipios = pd.read_csv(csv_idhm_municipios, encoding='latin-1', sep=',')
    # Criando conexão com sql
    conn = sql3.connect(db_file_path)
    cursor = conn.cursor()
    cursor.execute(f"DROP TABLE IF EXISTS {tabela_escolas}")
    cursor.execute(create_table_query)
    #print(df_escolas.isnull().sum())
    # Colocando o dataframe como tabela no banco de daos
    df_escolas.to_sql(tabela_escolas, conn, if_exists='append', index=False)
    #df_idhm_municipios.to_sql(tabela_idhm_municipios, conn, if_exists='replace', index=False)
    

finally:
    if 'conn' in locals() and conn:
        conn.close()


C:\Users\jeanb\AppData\Local\Temp\ipykernel_19312\3115616338.py:8: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_escolas = pd.read_csv(csv_escolas, encoding='latin-1', sep=';')


In [68]:
conn = sql3.connect("tp2.db")
cursor = conn.cursor()
query = "PRAGMA table_info(escola);"
df = pd.read_sql(query, conn)
display(df)

,cid,name,type,notnull,dflt_value,pk
0,0,NU_ANO,INTEGER,1,None,1
1,1,CO_UF_ESCOLA,INTEGER,0,None,0
2,2,SG_UF_ESCOLA,CHAR(2),0,None,0
3,3,CO_MUNICIPIO_ESCOLA,INTEGER,0,None,0
4,4,NO_MUNICIPIO_ESCOLA,TEXT,0,None,0
5,5,CO_ESCOLA_EDUCACENSO,INTEGER,0,None,2
6,6,NO_ESCOLA_EDUCACENSO,TEXT,0,None,0
7,7,TP_DEPENDENCIA_ADM_ESCOLA,CHAR(1),0,None,0
8,8,TP_LOCALIZACAO_ESCOLA,CHAR(1),0,None,0
9,9,NU_MATRICULAS,INTEGER,0,None,0
